In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import openrouteservice

In [13]:
place = "röthenbach an der pegnitz, Germany"
#place = "erlangen, Germany"
graph = ox.graph_from_place(place)

/opt/anaconda3/lib/python3.8/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


## getting the streets

In [14]:
streets = ox.graph_to_gdfs(graph)

## getting the buildings and the firestations in those buildings

In [15]:
tags = {"building": True}
buildings = ox.geometries_from_place(place,tags)
fire_stations = buildings[buildings.amenity == "fire_station"]

/opt/anaconda3/lib/python3.8/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [16]:
fire_station=[]
for i in range(len(fire_stations["geometry"])):
    fire_station.append(list(fire_stations["geometry"].exterior.iloc[i].coords)[1])

In [17]:
#for openrouteservice
key='5b3ce3597851110001cf624864c7b4d77462447c9035fc94c899056e'
coords = ((8.34234,48.23424),(8.34423,48.26424))

client = openrouteservice.Client(key=key)


/var/folders/wt/9gf59rdd2j16lhgptjnc2cxh0000gp/T/ipykernel_7711/1610083028.py:5: ResourceWarning: unclosed <ssl.SSLSocket fd=79, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.1.110', 58156), raddr=('129.206.5.53', 443)>
  client = openrouteservice.Client(key=key)


In [26]:
isochrones = []
map_isochrone = folium.Map(location=[fire_station[0][1],fire_station[0][0]], tiles='cartodbpositron', zoom_start=12)
style = [{'color': '#F7DC6F', 'weight':'1'},{'color': '#00FF00', 'weight':'2'},{'color': '#FF0000', 'weight':'2'},{'color': '#FF0000', 'weight':'2'}]
folium.features.GeoJson(buildings, name = 'buidlings', style_function=lambda x: style[0]).add_to(map_isochrone)
folium.features.GeoJson(streets,name = 'streets', style_function=lambda x: style[1]).add_to(map_isochrone)
folium.features.GeoJson(fire_stations,name = "Fire Stations",style_function=lambda x: style[2]).add_to(map_isochrone)
#folium.features.GeoJson(nodes,name = "nodes",style_function=lambda x: style[3]).add_to(map_isochrone)
 
for i, building  in enumerate(fire_station):
    lat, lon = building[0], building[1]
    coordinates = [[lat, lon]] # these have to be passed as lon and lat.
    
    isochrone = client.isochrones(locations= coordinates,

                                            profile="driving-car",
                                            range_type = 'time',
                                            
                                            range = [120,240,600])
    isochrones.append(isochrone)
    # add geojson to map with population
    #population = isochrone['features'][0]['properties']['total_pop']
    folium.GeoJson(isochrone, name='isochrone of fire_station_{}'.format(i+1)).add_to(map_isochrone)
folium.LayerControl().add_to(map_isochrone)
#save the 

In [27]:
#save the model
map_isochrone.save('isochrone.html')
#display the model as an image
#map_isochrone
